# Match Biblical names from multiple versions and across languages.
The JHU trabina project has done this with a list of 1128 names across 531 languages.

It would be good also to 'read' through the projects or extracts we have to do a similar matching.

We have some hand crafted data to get started: The Macula dataset and also the All Biblical Terms and Major Biblical Terms lists from Paratext.

In [1]:
#!/usr/bin/env python3

import csv
from collections import Counter
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import os
import pandas as pd
from pathlib import Path
import sys

In [2]:
def get_macula_df(file, sep="\t"):

    # Load the macula data. Note that the data has already been filtered for names-only using XBase.
    macula_df = pd.read_csv(file, dtype=str, sep=sep)
    macula_df.fillna('', inplace=True)
    
    #print(macula_df)
    
    #Index(['ref', 'Original unicode', 'Hebrew Original', 'Aramaic Original', 'Greek Original', 'Greek lemma', 'Greek normalized', 'Greek gloss', 'English gloss', 'Mandarin gloss']
    
    return macula_df

In [3]:
data_folder = Path("D:/GitHub/trabina/data") 
by_lang_folder = data_folder / "by-lang"
jhu_filename = "eng"
compare_col = "English gloss"
english_names_file = by_lang_folder / jhu_filename
updated_macula_data_tsv = data_folder / "updated_macula_names.tsv"

macula_data_tsv = data_folder / "macula_names.tsv"
macula_df = get_macula_df(macula_data_tsv)

simple_df = macula_df.drop(columns= ['ref','Hebrew Original', 'Aramaic Original', 'Greek Original'])
#Remove duplicate rows from the simple_df
simple_df.drop_duplicates(subset=None, keep='first', inplace=True)

print(f"\nThe simple macula dataframe has these columns:\n{simple_df.columns}")
simple_df


The simple macula dataframe has these columns:
Index(['Original unicode', 'Greek lemma', 'Greek normalized', 'Greek gloss',
       'English gloss', 'Mandarin gloss'],
      dtype='object')


,Original unicode,Greek lemma,Greek normalized,Greek gloss,English gloss,Mandarin gloss
0,יְהוָ֥ה,,,,LORD,耶和华
1,יְהוָ֤ה,,,,LORD,耶和华
2,יְהוָ֨ה,,,,LORD,耶和华
3,יְהוָ֧ה,,,κύριος,LORD,耶和华
4,עֵ֖דֶן,,,εδεμ,Eden,伊甸
...,...,...,...,...,...,...
38793,ΒΑΒΥΛΩΝ,Βαβυλών,ΒΑΒΥΛΩΝ,,Babylon,
38801,Ἰησοῦ,Ἰησοῦς,Ἰησοῦ,,,
38802,Χριστοῦ,Χριστός,Χριστοῦ,,,
38805,Γὼγ,Γώγ,Γώγ,,Gog,


In [4]:

data_folder = Path("D:/GitHub/trabina/data") 
by_lang_folder = data_folder / "by-lang"
jhu_filename = "eng"
compare_col = "English gloss"
english_names_file = by_lang_folder / jhu_filename
updated_macula_data_tsv = data_folder / "updated_macula_names.tsv"

macula_data_tsv = data_folder / "macula_names.tsv"
macula_df = get_macula_df(macula_data_tsv)

simple_df = macula_df.drop(columns= ['ref','Hebrew Original', 'Aramaic Original', 'Greek Original'])
#Remove duplicate rows from the simple_df
simple_df.drop_duplicates(subset=None, keep='first', inplace=True)

print(f"\nThe simple macula dataframe has these columns:\n{simple_df.columns}")
simple_df


The simple macula dataframe has these columns:
Index(['Original unicode', 'Greek lemma', 'Greek normalized', 'Greek gloss',
       'English gloss', 'Mandarin gloss'],
      dtype='object')


,Original unicode,Greek lemma,Greek normalized,Greek gloss,English gloss,Mandarin gloss
0,יְהוָ֥ה,,,,LORD,耶和华
1,יְהוָ֤ה,,,,LORD,耶和华
2,יְהוָ֨ה,,,,LORD,耶和华
3,יְהוָ֧ה,,,κύριος,LORD,耶和华
4,עֵ֖דֶן,,,εδεμ,Eden,伊甸
...,...,...,...,...,...,...
38793,ΒΑΒΥΛΩΝ,Βαβυλών,ΒΑΒΥΛΩΝ,,Babylon,
38801,Ἰησοῦ,Ἰησοῦς,Ἰησοῦ,,,
38802,Χριστοῦ,Χριστός,Χριστοῦ,,,
38805,Γὼγ,Γώγ,Γώγ,,Gog,


In [5]:
def get_name_matrix(folder):
    
    all_names = dict()
    
    folder = Path(folder)
    files = sorted(folder.glob(r'*'))
    #print([file.name[0:3] for file in files])
    
    for file in files:
        with open(file, 'r', encoding='utf-8') as fin:
            names = [name.strip('\n').title() for name in fin.readlines()]
            all_names[file.name] = names   
    
    return all_names

In [6]:
#Get all the names and make a dataframe
all_jhu_names = get_name_matrix(by_lang_folder)
jhu_df = pd.DataFrame.from_dict(all_jhu_names, dtype=str)

print(f"\nThe JHU dataframe has these columns:\n {jhu_df.columns}")  
jhu_df



The JHU dataframe has these columns:
 Index(['aai_aai', 'aak_aak', 'aau_aau', 'abt_maprik', 'aby_aby', 'acd_acd',
       'ace_ace', 'acf_acf', 'acn_acn', 'acr_cubulcu',
       ...
       'tpa_tpa', 'tpi_tpi', 'tpm_tpm', 'tsn_1908', 'tur_2009', 'ukr_1871',
       'urd_arabic', 'vie_1926compounds', 'xho_1996', 'zul_zul'],
      dtype='object', length=592)


,aai_aai,aak_aak,aau_aau,abt_maprik,aby_aby,acd_acd,ace_ace,acf_acf,acn_acn,acr_cubulcu,...,tpa_tpa,tpi_tpi,tpm_tpm,tsn_1908,tur_2009,ukr_1871,urd_arabic,vie_1926compounds,xho_1996,zul_zul
0,Aaron,Erono,Aron,Eron,Eroni,Aron,Harun,Éronn,Aron,Aaron,...,-,Aron,Aarɔn,Arone,Harun,Аарон,ہارون,A-Rôn,Uaron,Ku-Aroni
1,Abaddon,Abadonoyɨ,Abadon,Abadon,Abadoni,Abadon,Abadon,Abadonn,Abadon,Abadon,...,-,Abadon,Abadɔn,Abatone,Abadon,Авадон,ابدون,A-Ba-Đôn,Uapoliyon,Lingu-Abadoni
2,-,-,-,-,-,-,-,-,-,-,...,-,Abarim,Abarim,-,-,-,-,A-Ba-Rim,,-
3,Abba,Ápe,Abba,Wao,Ufane,Sɛi,Oe,Papa,Aba,Chawesaj,...,Ama'U,Aba,Kote,Aba,Abba,Авва,ابّا,A-Ba,Tata,Aba
4,-,-,-,-,-,-,-,-,-,-,...,-,Apdon,Abdɔn,-,-,-,-,Áp-Đôn,Uabdon,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,-,-,-,-,-,-,-,-,-,-,...,-,Soba,Zoba,-,-,-,-,Xô-Ba,Wasezobha,-
1125,-,-,-,-,-,-,-,-,-,-,...,-,Sora,Zora,-,-,-,-,Xô-Ra,Ezora,-
1126,Zerubbabel,Serababero,Serubabel,Serababel,Serubabeo,Serubabelɛ,Zerubabel,Zèròbabèl,Zerubabe,Zorobabel,...,-,Serubabel,Zɛrubabɛl,Serubabele,Zerubabel,Заровавель,زرُبابل,Xô-Rô-Ba-Bên,Uzerubhabheli,Uzorobabeli
1127,-,-,-,-,-,-,-,-,-,-,...,-,Suf,Zuf,-,-,-,-,Xu-Phơ,Kazufi,-


In [7]:
#jhu_df = jhu_df.set_index(["eng"], drop=True, append=False, inplace=False, verify_integrity=False)
jhu_df.describe()

,aai_aai,aak_aak,aau_aau,abt_maprik,aby_aby,acd_acd,ace_ace,acf_acf,acn_acn,acr_cubulcu,...,tpa_tpa,tpi_tpi,tpm_tpm,tsn_1908,tur_2009,ukr_1871,urd_arabic,vie_1926compounds,xho_1996,zul_zul
count,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
unique,483,472,481,465,456,478,470,491,471,482,...,96,974,953,488,425,492,480,988,1006,493
top,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
freq,556,573,549,576,583,558,568,553,551,555,...,1006,33,70,548,619,551,554,26,50,568


Not sure that merging like this is useful.

In [8]:
#Concatenate the two dataframes joining on exact matches of 
# 'English gloss' and jhu_eng columns. Retain both.

merged_df = pd.merge(jhu_df, simple_df, how='inner', left_index=True,right_on='English gloss', indicator=True)
merged_df.rename(columns={"_merge": "matched_on_eng"},inplace=True)
merged_df

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
jhu_df.index
#jhu_df['eng']


In [ ]:
# Read in the All terms data from silnlp
sil_assets_path = Path('D:/GitHub/silnlp/silnlp/assets')
all_terms_file = sil_assets_path / 'All-metadata.txt'

all_terms = pd.read_table(all_terms_file,header=None, usecols=[0]).squeeze("columns")
all_terms.rename('terms')

In [ ]:
# Split into words
terms_df = all_terms.str.split(' ', expand=True)
terms_df

In [ ]:
terms_df.rename({0: "term", 1: "note"}, axis="columns", inplace=True)
terms_df['AR'] = terms_df['note'] == '(AR)'
terms_df['DC'] = terms_df['note'] == '(DC)'
terms_df.drop(columns=['note'],inplace=True)
terms_df

In [ ]:
# Read in the Major terms data from silnlp
sil_assets_path = Path('D:/GitHub/silnlp/silnlp/assets')
major_terms_file = sil_assets_path / 'Major-metadata.txt'

major_terms = pd.read_table(major_terms_file,header=None)
major_terms.rename({0: "term", 1: "domain", 2:'category'}, axis="columns", inplace=True)
major_terms[['term', 'note']] = major_terms['term'].str.split(' ', 1, expand=True)

major_terms[['term', 'sense']] = major_terms['term'].str.split('-', 1, expand=True)
major_terms['AR'] = major_terms['note'] == '(AR)'
major_terms['DC'] = major_terms['note'] == '(DC)'
major_terms.drop(columns=['note'],inplace=True)

major_terms

In [ ]:
major_terms.to_csv(r"D:\GitHub\davidbaines\trabina\data\major_terms.txt", sep = '\t')

In [ ]:
#How many of the 8648 major terms are in the All terms data?
major_terms['all_terms_exact']=major_terms['term'].map(terms_df['term'].value_counts())
major_terms['all_terms_exact'] = major_terms['all_terms_exact'].fillna(0)
major_terms.sort_values('all_terms_exact',ascending=False)

In [ ]:
# These major terms aren't found exactly in the All-metadata.
# major_terms.loc[major_terms['all_terms_exact'].isna()]
major_terms.loc[major_terms['all_terms_exact'] == 0]

In [ ]:
col = major_terms['all_terms_exact'] 
count = col[col != 0].count()
print(f"There are {count} major-metadata terms that appear exactly in the All metadata file.")
print(f"There are {len(major_terms) - count} major-metadata terms that don't appear exactly in the All metadata file.")


In [ ]:
def count_matches(reference_col, source_col):
    matches = source_col.map(reference_col.value_counts()).fillna(0).astype(int)
    # Return count of values greater than 0 
    return matches[matches > 0].count()

#This is very slow.
#def find_matches(reference_col, source_col):
#    matches = [source for source in source_col if source in reference_col.unique()]
#    return matches

# This is also slow.
def find_matches(reference_col, source_col):
    matches = source_col.map(reference_col.value_counts()).fillna(0).astype(int)
    # Return values greater than 0 
    return matches[matches > 0]

# This is almost instant.
def find_matches(reference_col, source_col):
    return set(source_col).intersection(set(reference_col))

def report_matches(ref_df, ref_columns, search_dict):
    
    for name, col in search_dict.items():
        all_matches = set()
        match_count = 0
        unique_values = col.unique()
        print(f"Searching for the {len(unique_values)} '{name}' in the .")
        for ref_column in ref_columns:
            matches = find_matches(ref_df[ref_column],unique_values)
            match_count += len(matches)
            print(f"There are {len(matches)} found in the Macula '{ref_column}' column.")
            all_matches = all_matches.union(matches)

        print(f"{match_count} '{name}' matched of which {len(all_matches)} are unique.")
        print(sorted(all_matches))
        print('\n')

In [ ]:
# Count how many terms from All, Major and JHU (eng) occur in the Macula data.
# Macula Index(['Original unicode', 'Greek lemma', 'Greek normalized', 'Greek gloss', 'English gloss', 'Mandarin gloss']

terms_macula         = simple_df['Original unicode']
terms_macula_english = simple_df['English gloss']

terms_all     = terms_df['term']
terms_major   = major_terms['term']
terms_jhu_eng = jhu_df['eng']

#print(f"There are {count_matches(terms_macula,terms_all)}   all_terms out of {len(terms_all)} found in the Macula 'Original unicode' column.")
#print(f"There are {count_matches(terms_macula,terms_major)} major_terms out of {len(terms_major)} found in the Macula 'Original unicode' column.")
#print(f"There are {count_matches(terms_macula_english,terms_jhu_eng)} jhu_eng terms out of {len(terms_jhu_eng)} found in the Macula 'Original unicode' column.")

# How many of these Original language terms are found exactly as a key in other lists?
macula_search_columns = ['Original unicode', 'Hebrew Original', 'Aramaic Original', 'Greek Original', 'Greek lemma', 'Greek normalized', 'Greek gloss']
search = {'All terms':terms_all, 'Major terms':terms_major}
report_matches(macula_df, macula_search_columns, search)


In [ ]:
macula_search_columns = ['English gloss']
search = {'JHU eng terms':terms_jhu_eng}
report_matches(macula_df, macula_search_columns, search)

macula_search_columns = ['Original unicode', 'Greek Original', 'Greek lemma', 'Greek normalized', 'Greek gloss']
search = {'JHU grc_accented_terms' : jhu_df['grc_accented'], 'JHU ell_helenic1 terms' : jhu_df['ell_hellenic1']}
report_matches(macula_df, macula_search_columns, search)




In [ ]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [ ]:
def count_matches(reference_col, source_col):
    str2Match = source_col.fillna('').tolist()
    strOptions = reference_col.fillna('').tolist()
    
    matches = source_col.map(reference_col.value_counts()).fillna(0).astype(int)
    # Return count of values greater than 0 
    return matches[matches > 0].count()

In [ ]:
major_matches = terms_major.map(terms_macula.value_counts()).fillna(0).astype(int)
# Return count of values greater than 0 
print(major_matches[major_matches > 0].count())
print(major_matches)

#all_name_match,   all_ratio_match = checker(terms_macula,terms_all[0:100])
#major_name_match, major_ratio_match=checker(terms_macula,terms_major)

#terms_df['fuzzy_match']=pd.Series(all_name_match)
#terms_df['fuzzy_ratio']=pd.Series(all_ratio_match)
#print(all_name_match,all_ratio_match)

In [ ]:
#print(len(all_name_match),len(all_ratio_match))
#all_fuzzy_matches = pd.DataFrame.from_dict({'Original unicode': terms_macula, 'all_metadata fuzzy match' : all_name_match, 'all_metadata fuzzy ratio' :all_ratio_match})
#all_fuzzy_matches

In [ ]:
def get_terms_from_files(folder,filenames):
    
    all_names = dict()
    
    folder = Path(folder)
    files = [folder / filename for filename in filenames]
    #print([file.name[0:3] for file in files])
    
    for file in files:
        with open(file, 'r', encoding='utf-8') as fin:
            names = [name.strip('\n').title() for name in fin.readlines()]
            all_names[file.name] = names   
    
    return all_names

In [ ]:
# Read in the Major terms metadata from silnlp
sil_assets_path = Path('D:/GitHub/silnlp/silnlp/assets')
source_major_terms_file = sil_assets_path / 'Major-metadata.txt'

source_major_terms = pd.read_table(source_major_terms_file,header=None)
source_major_terms.rename({0: "term", 1: "domain", 2:'category'}, axis="columns", inplace=True)
source_major_terms[['term', 'note']] = source_major_terms['term'].str.split(' ', 1, expand=True)

source_major_terms[['term', 'sense']] = source_major_terms['term'].str.split('-', 1, expand=True)
source_major_terms['AR'] = source_major_terms['note'] == '(AR)'
source_major_terms['DC'] = source_major_terms['note'] == '(DC)'
source_major_terms.drop(columns=['note'],inplace=True)

source_major_terms

In [ ]:
other_major_terms_files = [
'en-Major-glosses.txt',
'es-Major-glosses.txt',
'fr-Major-glosses.txt',
'id-Major-glosses.txt']

names = [file[:-4].replace('-','_') for file in other_major_terms_files]
new_col_names = {file: file[:-4].replace('-','_').replace('_glosses','') for file in other_major_terms_files}
print(new_col_names)


In [ ]:
# https://stackoverflow.com/questions/28218698/how-to-iterate-over-columns-of-pandas-dataframe-to-run-regression
# for idx, row in df.iterrows() iterates over rows
# df.iteritems() does the same for columns,
# and df.items() may be much the same.
# Beware df[column] can be a DataFrame or a Serie if you have columns with duplicated names.

other_major_terms_data = get_terms_from_files(sil_assets_path,other_major_terms_files)
other_major_terms = pd.DataFrame.from_dict(other_major_terms_data, dtype=str)
other_major_terms.rename(new_col_names, axis="columns", inplace=True)
other_major_terms

In [ ]:
for column in other_major_terms:
    print(column)
    other_major_terms[column] = other_major_terms[column].str.split(('\t'), expand=False)

other_major_terms


In [ ]:
major_metadata = pd.concat([source_major_terms, other_major_terms], axis=1)
major_metadata

In [ ]:
# How many of these Original language terms are found exactly as a key in other lists?
simple_df
possible_match_columns = ['Original unicode', 'Hebrew Original', 'Aramaic Original', 'Greek Original', 'Greek lemma', 'Greek normalized', 'Greek gloss']
